In [1]:
import pandas as pd

In [25]:
data = pd.read_csv('data/EC10.csv')

In [26]:
data.head()


,id,text,date,company,sector
0,32934,"Zoe's Kitchen, Inc. (NYSE:ZOES)\nQ2 2015 Earni...",2015-08-28 00:11:00,"Zoe's Kitchen, Inc.",Consumer Services
1,32905,"Autodesk, Inc. (NASDAQ:ADSK)\nQ2 2016 Earnings...",2015-08-28 00:07:00,"Autodesk, Inc.",Technology
2,32926,Smith & Wesson Holding Corp. (NASDAQ:SWHC)\nQ1...,2015-08-27 23:25:00,Smith & Wesson Holding Corporation,Capital Goods
3,32930,"Ulta Salon, Cosmetics & Fragrance, Inc. (NASDA...",2015-08-27 23:09:00,"Ulta Salon, Cosmetics & Fragrance, Inc.",Consumer Services
4,32907,Splunk Inc. (NASDAQ:SPLK)\nQ2 2016 Earnings Co...,2015-08-27 22:56:00,Splunk Inc.,Technology


1- Which is the most verbose sector (uses more words than others)

In [15]:
#data['words']=data['text'].apply(lambda  x: len(x.split()))

In [33]:
data['words']=data['text'].str.split().str.len()

In [34]:
data.groupby('sector')['words'].mean().sort_values().index[-1]

'Technology'

2- count the number of companies in each stock exchange

In [35]:
txt = data.iloc[0, 1]

In [37]:
import re

In [44]:
exp.findall(txt)[0]

('NYSE', 'ZOES')

In [45]:
def extract_txt(txt):
    exp = re.compile("\(([A-Z]+):([A-Z]+)\)")
    result = exp.findall(txt)
    if result:
        return result[0][0]

In [46]:
extract_txt(txt)

'NYSE'

In [48]:
data['stock'] = data['text'].apply(extract_txt)
data.head()

,id,text,date,company,sector,words,stock
0,32934,"Zoe's Kitchen, Inc. (NYSE:ZOES)\nQ2 2015 Earni...",2015-08-28 00:11:00,"Zoe's Kitchen, Inc.",Consumer Services,7690,NYSE
1,32905,"Autodesk, Inc. (NASDAQ:ADSK)\nQ2 2016 Earnings...",2015-08-28 00:07:00,"Autodesk, Inc.",Technology,11259,NASDAQ
2,32926,Smith & Wesson Holding Corp. (NASDAQ:SWHC)\nQ1...,2015-08-27 23:25:00,Smith & Wesson Holding Corporation,Capital Goods,7015,NASDAQ
3,32930,"Ulta Salon, Cosmetics & Fragrance, Inc. (NASDA...",2015-08-27 23:09:00,"Ulta Salon, Cosmetics & Fragrance, Inc.",Consumer Services,11173,NASDAQ
4,32907,Splunk Inc. (NASDAQ:SPLK)\nQ2 2016 Earnings Co...,2015-08-27 22:56:00,Splunk Inc.,Technology,10612,NASDAQ


In [49]:
data.groupby('stock')['company'].count()

stock
NASDAQ    6
NYSE      4
Name: company, dtype: int64

In [53]:
data['stock'] =  data['text'].str.findall("\(([A-Z]+):([A-Z]+)\)").str.get(0).str.get(0)
data.groupby('stock')['company'].count()

stock
NASDAQ    6
NYSE      4
Name: company, dtype: int64

In [56]:
data['text'].str.extract("\(([A-Z]+):([A-Z]+)\)").groupby(0).count()

,1
0,
NASDAQ,6
NYSE,4


3- What are the top 10 most frequent words?

In [60]:
def count_words(txt):
    res = {}
    words = txt.split()
    for word in set(words):
        res[word] = 0
    for word in words:
        res[word] += 1
    return res

In [86]:
def top_freq(word_count, n=10):
    swc = sorted(word_count.items(), key=lambda x: -x[1])
    return dict(swc[:n])

In [96]:
top_freq(count_words((data['text'] + " ").sum()))

{'the': 3552,
 'to': 2324,
 'of': 2245,
 'and': 2166,
 'in': 1615,
 'that': 1356,
 'a': 1347,
 'our': 1201,
 'we': 1166,
 'is': 892}

4- What are the top 10 most frequent words per sector?

In [100]:
def freq_of_txt_series(series, n = 10):
    return top_freq(count_words((series + " ").sum()), n=n)

In [105]:
data.groupby('sector')['text'].apply(freq_of_txt_series).unstack()

,the,of,and,to,in,our,that,a,-,we,I,is
sector,,,,,,,,,,,,
Capital Goods,253.0,170.0,153.0,151.0,124.0,121.0,101.0,95.0,95.0,84.0,NaN,NaN
Consumer Services,1238.0,813.0,767.0,900.0,562.0,428.0,474.0,494.0,344.0,421.0,NaN,NaN
Finance,363.0,183.0,283.0,245.0,207.0,103.0,163.0,134.0,NaN,146.0,104.0,NaN
Technology,1698.0,1079.0,963.0,1028.0,722.0,549.0,618.0,624.0,NaN,515.0,NaN,471.0


In [106]:
from collections import Counter

In [114]:
def top_10(s, n=10):
    text = (s + " ").sum().split()
    count = Counter(text)
    return pd.Series(count).sort_values(ascending=False)[:n]

In [117]:
data.groupby('sector')['text'].apply(top_10).unstack()

,-,I,a,and,in,is,of,our,that,the,to,we
sector,,,,,,,,,,,,
Capital Goods,95.0,NaN,95.0,153.0,124.0,NaN,170.0,121.0,101.0,253.0,151.0,84.0
Consumer Services,344.0,NaN,494.0,767.0,562.0,NaN,813.0,428.0,474.0,1238.0,900.0,421.0
Finance,NaN,104.0,134.0,283.0,207.0,NaN,183.0,103.0,163.0,363.0,245.0,146.0
Technology,NaN,NaN,624.0,963.0,722.0,471.0,1079.0,549.0,618.0,1698.0,1028.0,515.0
